# Training

In [1]:
import pandas as pd
import torch
from pytorch_lightning import Trainer
from sklearn.model_selection import train_test_split
from torchvision import transforms

from src.colors import bcolors
from config import Config
from src.training.data import EuroSatDataModule

c = bcolors()
config = Config()

In [2]:
from src.datasets.EuroSatMS import EuroSatMS

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

df = pd.read_csv(config.TRAIN_FILE)

train_df, val_df = train_test_split(df, test_size=0.15, stratify=df['label'], random_state=67)

chan = [3, 2, 1]

data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

data_augmentations = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(90),
    transforms.RandomRotation(90),
    transforms.RandomRotation(90),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
])

ds_train = EuroSatMS(train_df, config.TRAIN_MS_DIR, select_chan=chan, transform=data_transform, augment=data_augmentations)
ds_val = EuroSatMS(val_df, config.TRAIN_MS_DIR, select_chan=chan, transform=data_transform, augment=data_augmentations)


Preloading images...
Number of images: 22950
Number of jobs:   -4 

Preloading images...
Number of images: 4050
Number of jobs:   -4 


In [3]:
BATCH_SIZE = 256

print(f"""{c.OKGREEN}Initializing the data module...{c.ENDC}""")
data_module = EuroSatDataModule(ds_train, ds_val, config.TRAIN_MS_DIR, BATCH_SIZE)

Initializing the data module...


In [4]:
from src.training.cnn import LitEuroSatCnn
from src.training.resNet50 import EuroSatResNet

LEARNING_RATE = 0.01
KERNEL_SIZE = 3
N_CLASSES = 10

print(f"""{c.OKGREEN}Initializing the model...{c.ENDC}""")
lightning_model = EuroSatResNet(num_classes=N_CLASSES, learning_rate=LEARNING_RATE)
# lightning_model = LitEuroSatCnn(num_classes=10, learning_rate=LEARNING_RATE, num_channels=len(chan), kernel_size=KERNEL_SIZE)

Initializing the model...


### Train ResNet50
First the pretrained model is frozen and trained for n epochs.

In [8]:
N_EPOCHS = 5

# Initialize the Trainer
trainer = Trainer(max_epochs=N_EPOCHS, accelerator="gpu", devices=1, log_every_n_steps=2)

# Train the model
trainer.fit(lightning_model, datamodule=data_module)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | ResNet           | 23.5 M
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
20.5 K    Trainable params
23.5 M    Non-trainable params
23.5 M    Total params
94.114    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.


Unfreeze the last n_layer, set a smaller lr and train for n epochs.

In [ ]:
# Adjust the learning rate and max epochs
NEW_LR = 1e-4
trainer.max_epochs = 3
lightning_model.adjust_learning_rate(NEW_LR)

# Unfreeze and train the model
lightning_model.unfreez_children_layers(2)
trainer.fit(lightning_model, datamodule=data_module)

Unfreeze the last n_layer, set a smaller lr and train for n epochs.

In [ ]:
# Adjust the learning rate and max epochs
NEW_LR = 1e-4
trainer.max_epochs = 2
lightning_model.adjust_learning_rate(NEW_LR)

# Unfreeze and train the model
lightning_model.unfreez_children_layers(4)
trainer.fit(lightning_model, datamodule=data_module)

In [ ]:
# Test the model
trainer.validate(lightning_model, datamodule=data_module)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

# Predict on the test set

In [ ]:
from src.training.resNet50 import EuroSatResNet
from src.training.cnn import LitEuroSatCnn
from config import Config

config = Config()

# Load the model from the checkpoint
# model = EuroSatResNet.load_from_checkpoint("lightning_logs/version_1/checkpoints/epoch=4-step=425.ckpt")
model = LitEuroSatCnn.load_from_checkpoint("lightning_logs/version_9/checkpoints/epoch=9-step=900.ckpt")

# Set the model to evaluation mode
model.eval()
model.freeze()

In [ ]:
# Test the model
trainer.validate(model, datamodule=data_module)

In [ ]:
from torchvision import transforms
from src.datasets.EuroSatTest import EuroSatTestSet
from torch.utils.data import DataLoader

chan = [6, 5, 4, 3, 2, 1]
dataset = EuroSatTestSet(config.TEST_MS_DIR, select_chan=chan)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False)


In [ ]:
import torch
import numpy as np


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

N_CLASSES = 10
categorys = dataset.enc.categories_[0]

predictions = []
probabilities = []
ohe = []
images = []
sample_ids = []

with torch.no_grad():
    for batch in dataloader:
        inputs, samp_id = batch
        inputs = inputs.to(device)

        outputs = model(inputs)

        probs = torch.softmax(outputs, dim=1).cpu()
        
        _, preds = torch.max(probs, 1)
        preds = preds.cpu().numpy()
        
        preds_enc = np.zeros((preds.size, N_CLASSES))
        preds_enc[np.arange(preds.size), preds] = 1
        
        pred_labels = np.array([categorys[p] for p in preds])
        
        
        # print(preds[0])
        # print(preds_enc[0])
        # print(pred_labels[0])
        
        predictions.extend(pred_labels)
        images.extend(inputs.cpu())
        sample_ids.extend(samp_id.cpu())
        probabilities.extend(probs.cpu())
        ohe.extend(preds_enc)


In [ ]:
import pandas as pd


sub_df = pd.DataFrame({'test_id': np.array(sample_ids), 'label': np.array(predictions)})
sub_df = sub_df.sort_values(by='test_id')
print(sub_df.head())

sub_df.to_csv('submission.csv', index=False)

In [ ]:
from PIL import Image
from matplotlib import pyplot as plt

for img, samp_id, pred, probs, e in zip(images[:100], 
                                     sample_ids[:100], 
                                     predictions[:100], 
                                     probabilities[:100],
                                     ohe[:1000]):
    # if pred != "Industrial":
    #     continue
    print(f"Sample ID: {samp_id}, Prediction: {pred}")
    print(e)
    print(pred)
    raw = np.load(f"data/test/NoLabel/test_{samp_id}.npy")
    img = raw[:, :, [3, 2, 1]]
    print(img.shape)
    rgb_min = img.min()
    rgb_max = img.max()

    img = (img - rgb_min) / (rgb_max - rgb_min)
    
    plt.imshow(img)
    plt.show()